In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np

In [3]:
data = pd.read_csv("master_dataset/processed_data.csv")
list(data.columns)
# drop unwanted features


data = data.drop(['title', 'text', 'text_without_stopwords', 'title_without_stopwords','syllables', 
                  'polarity_category', 'overall_content', 'polarity_category_Neutral' , 'polarity_category_Positive',
                  'text_average_word_length',  'title_average_word_length',  'text_punctuation_count',
                    'title_punctuation_count',  'text_stopwords_count',  'title_stopwords_count',], axis=1)

#'title_word_count', 'title_sentence_count', 'title_average_word_length','title_punctuation_count', 'title_stopwords_count'  
# 'polarity'

In [4]:
#dataset is slightly imbalanced so we will perform upsampling to balance the dataset.
data['class'].value_counts()

0    21196
1    17462
Name: class, dtype: int64

In [5]:
list(data.columns)

['class',
 'text_word_count',
 'title_word_count',
 'text_sentence_count',
 'title_sentence_count',
 'flesch_readability',
 'subjectivity',
 'polarity',
 'Topic 1 Probability',
 'Topic 2 Probability',
 'Topic 3 Probbility',
 'Topic 4 Probability',
 'Topic 5 Probability']

In [6]:
data.describe()

,class,text_word_count,title_word_count,text_sentence_count,title_sentence_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000
mean,0.451705,411.374515,12.161209,15.050830,1.067877,0.006674,0.402753,0.056997,0.190953,0.324903,0.125233,0.185360,0.173552
std,0.497669,322.881353,3.765686,11.985707,0.270277,1.003826,0.124806,0.105170,0.281023,0.312565,0.210573,0.258829,0.234861
min,0.000000,1.000000,1.000000,1.000000,1.000000,-83.616811,0.000000,-1.000000,0.000376,0.000246,0.000120,0.000303,0.000268
25%,0.000000,220.000000,10.000000,7.000000,1.000000,-0.169506,0.337127,0.000000,0.002949,0.010969,0.001305,0.003113,0.003024
50%,0.000000,376.000000,11.000000,13.000000,1.000000,0.050549,0.405745,0.054757,0.030471,0.231605,0.006765,0.056382,0.056373
75%,1.000000,522.000000,14.000000,20.000000,1.000000,0.267479,0.475000,0.108144,0.282926,0.584998,0.157612,0.272065,0.268291
max,1.000000,8436.000000,45.000000,321.000000,4.000000,2.614284,1.000000,1.000000,0.997819,0.998037,0.996771,0.998911,0.998767


In [7]:
#first split the dataset into training and test sets
x = data.iloc[:,1:]
y = data.iloc[:,:1]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state = 4222)


x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.25, random_state = 4222)


#balance x_train with oversampling
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
# oversample = RandomOverSampler(sampling_strategy = 1)
x_train,y_train = undersample.fit_resample(x_train, y_train)
data = pd.concat([x_train,y_train],axis = 1)

#check that train set is oversampled
data['class'].value_counts()

0    10477
1    10477
Name: class, dtype: int64

In [8]:
# Since we will be performing SVC, and SVC works better on scaled data, we will be scaling all our to ensure that the model runs smoothly
# We will use the Min Max scaler

scaler = StandardScaler()

# Since polarity_category_Neutral and polarity_category_Positive are already from 0 to 1 scale, we will not have to further normalize it.
cols = data.iloc[:,0:-1].columns

# Normalize the data , since the data is not normally distributed we will use minmaxscaler
x_train[cols] = scaler.fit_transform(x_train[cols]) 

x_test[cols] = scaler.transform(x_test[cols])
x_validation[cols] = scaler.transform(x_validation[cols])



In [9]:
x_train.describe()

,text_word_count,title_word_count,text_sentence_count,title_sentence_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04
mean,-1.485984e-16,6.437250e-14,-1.452869e-16,-3.251644e-15,2.527858e-17,1.047745e-15,3.842122e-17,2.980446e-16,-7.839990e-16,-9.346081e-17,-7.456413e-17,5.200415e-16
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00
min,-1.265605e+00,-2.931249e+00,-1.181811e+00,-2.599167e-01,-8.144205e+01,-3.257379e+00,-9.876762e+00,-6.496807e-01,-1.083736e+00,-5.955218e-01,-7.069370e-01,-7.531941e-01
25%,-5.730816e-01,-6.205526e-01,-6.757172e-01,-2.599167e-01,-1.882355e-01,-5.262506e-01,-5.364375e-01,-6.405202e-01,-9.950502e-01,-5.901131e-01,-6.963361e-01,-7.411363e-01
50%,-1.113991e-01,-1.070645e-01,-1.696237e-01,-2.599167e-01,4.439102e-02,2.714473e-02,-2.145563e-02,-5.656766e-01,-2.508576e-01,-5.576516e-01,-4.995789e-01,-4.908026e-01
75%,3.318160e-01,6.631676e-01,4.208187e-01,-2.599167e-01,2.727143e-01,5.783038e-01,4.809640e-01,2.483683e-01,8.481717e-01,1.572635e-01,3.226032e-01,4.335337e-01
max,2.469634e+01,8.365489e+00,2.580984e+01,1.040515e+01,2.428584e+00,4.736168e+00,8.803887e+00,2.987980e+00,2.095495e+00,4.204596e+00,3.225915e+00,3.478053e+00


Linear SVM

In [10]:
# Linear SVM baseline model

linearSVC = SVC(kernel='linear',random_state=4222)
linearSVC.fit(x_train, np.ravel(y_train))
y_predval_linearSVC = linearSVC.predict(x_validation)

#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_linearSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_linearSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_linearSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_linearSVC))
print("-------------------------------")


#test metrics
y_pred_linearSVC = linearSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_pred_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_pred_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_pred_linearSVC))

Performance on Validation set:
Accuracy: 0.911148473874806
Precision: 0.9011744485820682
Recall: 0.9019495412844036
F1_score: 0.9015618283421695
-------------------------------
Performance on  Test set:
Accuracy: 0.9084324883600621
Precision: 0.9022209402942025
Recall: 0.8944809837003146
F1_score: 0.8983342906375646


In [11]:
#hyperparameter tuning with gridsearch for SVM  

grid_params = {
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001]
    }

scorer = metrics.make_scorer(metrics.f1_score)

gridCV = GridSearchCV(SVC(kernel='linear',random_state=4222), param_grid = grid_params, cv = 5, scoring = scorer, n_jobs=-1)

gridCV.fit(x_train,np.ravel(y_train))

print("Best Hyper Parameters: ", gridCV.best_params_)

Best Hyper Parameters:  {'C': 0.1, 'gamma': 1}


In [13]:
finalSVC = SVC( kernel= 'linear', C= 0.1, gamma= 1, random_state = 4222)
finalSVC.fit(x_train, np.ravel(y_train))
y_predval_finalSVC = finalSVC.predict(x_validation)


#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_finalSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_finalSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_finalSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_finalSVC))
print("-------------------------------")


#test metrics
y_predtest_linearSVC = finalSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_predtest_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_predtest_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_predtest_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_predtest_linearSVC))

Performance on Validation set:
Accuracy: 0.9110191412312467
Precision: 0.9016064257028112
Recall: 0.9010894495412844
F1_score: 0.9013478634929739
-------------------------------
Performance on  Test set:
Accuracy: 0.9081738230729436
Precision: 0.9019325064897606
Recall: 0.8941950243065485
F1_score: 0.8980470993681793
